<div style="font-size:38px;color:green"><u>Boltzmann Machine (BM)</u></div><br/><br/>
<sub>**</sub>

In [18]:
import numpy as np
import pandas as pd
# importing libraries
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [24]:
# importing and preprocessing data

movies = pd.read_csv('ml-1m/movies.dat',sep = '::',header = None, engine = 'python', encoding = 'latin-1')
users =  pd.read_csv('ml-1m/users.dat',sep = '::',header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat',sep = '::',header = None, engine = 'python', encoding = 'latin-1')

# training and test set

train = pd.read_csv('ml-100k/u1.base',delimiter='\t')
train = np.array(train,dtype='int')
test = pd.read_csv('ml-100k/u1.base',delimiter='\t')
test = np.array(test,dtype='int')


In [20]:
# getting max number of movies and users
no_users = int(max(max(train[:,0]),max(test[:,0])))
no_movies = int(max(max(train[:,1]),max(test[:,1])))

# converting data into an array
def convert(data):
    new_data = []
    for id_users in range(1, no_users + 1):
        id_movies = data[:,1][data[:,0] == id_users]
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(no_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data
train = convert(train)
test = convert(test)

In [21]:
# converting data into Torch tensor
train = torch.FloatTensor(train)
test = torch.FloatTensor(test)

# converting raiting into binary ratings 0,1
train[train == 0] = -1
train[train == 1] = 0
train[train == 2] = 0
train[train > 2] = 1
test[test == 0] = -1
test[test == 1] = 0
test[test == 2] = 0
test[test > 2] = 1

# Creating the architecture of the Neural Network
class RBM():
    def __init__(self, nv, nh):
        self.W = torch.randn(nh, nv)
        self.a = torch.randn(1, nh)
        self.b = torch.randn(1, nv)
    def sample_h(self, x):
        wx = torch.mm(x, self.W.t())
        activation = wx + self.a.expand_as(wx)
        p_h_given_v = torch.sigmoid(activation)
        return p_h_given_v, torch.bernoulli(p_h_given_v)
    def sample_v(self, y):
        wy = torch.mm(y, self.W)
        activation = wy + self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)
    def train(self, v0, vk, ph0, phk):
        self.W += torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)
        self.b += torch.sum((v0 - vk), 0)
        self.a += torch.sum((ph0 - phk), 0)
nv = len(train[0])
nh = 100
batch_size = 100
rbm = RBM(nv, nh)

In [23]:
# Training the RBM
nb_epoch = 10
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(0, no_users - batch_size, batch_size):
        vk = train[id_user:id_user+batch_size]
        v0 = train[id_user:id_user+batch_size]
        ph0,_ = rbm.sample_h(v0)
        for k in range(10):
            _,hk = rbm.sample_h(vk)
            _,vk = rbm.sample_v(hk)
            vk[v0<0] = v0[v0<0]
        phk,_ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0]))
        s += 1.
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

RuntimeError: The size of tensor a (1682) must match the size of tensor b (100) at non-singleton dimension 1

In [ ]:
# Testing the RBM
test_loss = 0
s = 0.
for id_user in range(nb_users):
    v = train[id_user:id_user+1]
    vt = test[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.
print('test loss: '+str(test_loss/s))